In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# 올바른 파일 경로 확인
# Make sure this path is correct by copying it from your Google Drive in Colab
data_zip_path = '/content/drive/MyDrive/1차 플젝/건강데이터_키_몸무게_삭제.csv'

# CSV 파일 읽기
data = pd.read_csv(data_zip_path, encoding='cp949')

# 데이터 확인
data.head()

Mounted at /content/drive


,ID,날짜,나이,성별,운동 유형,지속 시간,강도,칼로리 소모,평균 심박수,수면 시간,스트레스 수준,매일 걸음수,체내 수분 함량,BMI,휴식 심박수,수축기 혈압,이완기 혈압,건강 상태,흡연 상태,체력 수준
0,1,2024-01-01,56,F,Dancing,41,Low,3.3,103,6.6,3,7128,1.5,19.6,69.5,110.7,72.9,NaN,Never,0.04
1,1,2024-01-04,56,F,Swimming,28,Low,2.9,102,8.1,7,7925,1.8,19.6,69.5,110.7,72.9,NaN,Never,0.07
2,1,2024-01-05,56,F,Swimming,21,Medium,2.6,126,6.2,7,7557,2.7,19.6,69.5,110.7,72.9,NaN,Never,0.09
3,1,2024-01-07,56,F,Weight Training,99,Medium,10.7,141,7.2,8,11120,2.6,19.6,69.5,110.7,72.9,NaN,Never,0.21
4,1,2024-01-09,56,F,Swimming,100,Medium,12.7,112,7.1,1,5406,1.5,19.6,69.5,110.7,72.9,NaN,Never,0.33


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 데이터 불러오기
file_path = '/content/drive/MyDrive/1차 플젝/건강데이터_키_몸무게_삭제.csv'
data = pd.read_csv(file_path, encoding='cp949')
# 삭제할 컬럼 리스트에 포함시켜 제거
columns_to_drop = ['날짜']  # 삭제하고 싶은 컬럼명 입력
data = data.drop(columns=columns_to_drop)

# 범주형 변수 원-핫 인코딩
data_encoded = pd.get_dummies(data, columns=['성별', '운동 유형', '강도', '건강 상태', '흡연 상태'], drop_first=True)

# ID 1500번 이하로 필터링
filtered_data = data_encoded[data_encoded['ID'] <= 1500]

# 입력과 출력 변수 정의
# Remove the columns that were one-hot encoded from the drop list
X_filtered = filtered_data.drop(['나이', '지속 시간', '칼로리 소모', '평균 심박수', '수면 시간', '스트레스 수준', '매일 걸음수', '체내 수분 함량', '휴식 심박수', '수축기 혈압', '이완기 혈압', '체력 수준'], axis=1)
y_filtered = filtered_data['BMI']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)

# 랜덤 포레스트 회귀 모델 (최적화된 트리 개수)
rf_model = RandomForestRegressor(n_estimators=50, random_state=42)
rf_model.fit(X_train, y_train)

# 예측 및 성능 평가
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}, R2: {r2}')

MSE: 5.334786259082194e-26, R2: 1.0


In [ ]:
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 데이터 불러오기
file_path = '/content/drive/MyDrive/1차 플젝/건강데이터_키_몸무게_삭제.csv'
data = pd.read_csv(file_path, encoding='cp949')
# 삭제할 컬럼 리스트에 포함시켜 제거
columns_to_drop = ['날짜']  # 삭제하고 싶은 컬럼명 입력
data = data.drop(columns=columns_to_drop)

# 범주형 변수 원-핫 인코딩
data_encoded = pd.get_dummies(data, columns=['성별', '운동 유형', '강도', '건강 상태', '흡연 상태'], drop_first=True)

# ID 기준으로 학습/테스트 데이터 분리
train_data = data_encoded[data_encoded['ID'] <= 2500]
test_data = data_encoded[(data_encoded['ID'] > 2500) & (data_encoded['ID'] <= 3000)]

# ['나이', '지속 시간', '칼로리 소모', '평균 심박수', '수면 시간', '스트레스 수준', '매일 걸음수', '체내 수분 함량', '휴식 심박수', '수축기 혈압', '이완기 혈압', '체력 수준'
# '지속 시간', '칼로리 소모', '스트레스 수준', '매일 걸음수', '체력 수준'
# 입력과 출력 변수 설정
X_train = train_data.drop(['BMI', 'ID', '나이', '휴식 심박수', '평균 심박수'], axis=1)
y_train = train_data['BMI']
X_test = test_data.drop(['BMI', 'ID', '나이', '휴식 심박수', '평균 심박수'], axis=1)
y_test = test_data['BMI']

# LightGBM 회귀 모델 훈련
lgbm_model = LGBMRegressor()
lgbm_model.fit(X_train, y_train)

# 예측 및 성능 평가
y_pred = lgbm_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}, R²: {r2}')


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029045 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1506
[LightGBM] [Info] Number of data points in the train set: 573289, number of used features: 26
[LightGBM] [Info] Start training from score 22.745896
MSE: 5.832832154050826, R²: 0.5046013560713176


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

# ... (your existing code for data loading and splitting) ...

# 범주형 변수 이름 가져오기
categorical_features = ['성별', '운동 유형', '강도', '건강 상태', '흡연 상태']

# Fill NaN values in categorical columns with a placeholder string like 'missing'
for col in categorical_features:
    X_train[col] = X_train[col].fillna('missing').astype(str)
    X_test[col] = X_test[col].fillna('missing').astype(str)

# 범주형 변수 인덱스 가져오기
cat_features_indices = [X_train.columns.get_loc(col) for col in categorical_features if col in X_train.columns]

# CatBoost 회귀 모델 설정 및 훈련
catboost_model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.1, verbose=200)
catboost_model.fit(X_train, y_train, cat_features=cat_features_indices)

# ... (rest of your code) ...

0:	learn: 3.5319776	total: 835ms	remaining: 13m 54s
200:	learn: 2.3289501	total: 2m 6s	remaining: 8m 21s
400:	learn: 1.8874626	total: 4m 19s	remaining: 6m 27s
600:	learn: 1.6481408	total: 6m 25s	remaining: 4m 16s
800:	learn: 1.4933702	total: 8m 33s	remaining: 2m 7s
999:	learn: 1.3865250	total: 10m 38s	remaining: 0us


In [ ]:
# 예측 및 성능 평가
y_pred = catboost_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
# 예측 및 성능 평가
y_pred = catboost_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}, R²: {r2}')

MSE: 2.80382423293135, R²: 0.7618634162404418


In [ ]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

# 데이터 불러오기
file_path = '/content/drive/MyDrive/1차 플젝/건강데이터_키_몸무게_삭제.csv'
data = pd.read_csv(file_path, encoding='cp949')

# 필요 없는 날짜 컬럼 삭제
columns_to_drop = ['날짜']
data = data.drop(columns=columns_to_drop)

# ID 기준으로 학습/테스트 데이터 분리
train_data = data[data['ID'] <= 2500]
test_data = data[(data['ID'] > 2500) & (data['ID'] <= 3000)]

# 입력과 출력 변수 설정
X_train = train_data.drop(['BMI', 'ID', '나이', '휴식 심박수', '평균 심박수'], axis=1)
y_train = train_data['BMI']
X_test = test_data.drop(['BMI', 'ID', '나이', '휴식 심박수', '평균 심박수'], axis=1)
y_test = test_data['BMI']

# 범주형 특성을 숫자형으로 변환 (Label Encoding)
categorical_features = ['성별', '운동 유형', '강도', '건강 상태', '흡연 상태']
for feature in categorical_features:
    le = LabelEncoder()
    # Fit the encoder on the training data and transform both train and test
    X_train[feature] = le.fit_transform(X_train[feature])
    X_test[feature] = le.transform(X_test[feature])


# XGBoost 회귀 모델 설정 및 훈련
xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=6, random_state=42)
xgb_model.fit(X_train, y_train)

# 예측 및 성능 평가
y_pred = xgb_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}, R²: {r2}')

MSE: 3.304404848128248, R²: 0.7193477135087414


In [ ]:
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# 데이터 불러오기
file_path = '/content/drive/MyDrive/1차 플젝/건강데이터_키_몸무게_삭제.csv'
data = pd.read_csv(file_path, encoding='cp949')

# 필요 없는 날짜 컬럼 삭제
columns_to_drop = ['날짜']
data = data.drop(columns=columns_to_drop)

# 범주형 변수 원-핫 인코딩
data_encoded = pd.get_dummies(data, columns=['성별', '운동 유형', '강도', '건강 상태', '흡연 상태'], drop_first=True)

# ID 기준으로 학습/테스트 데이터 분리
train_data = data_encoded[data_encoded['ID'] <= 2500]
test_data = data_encoded[(data_encoded['ID'] > 2500) & (data_encoded['ID'] <= 3000)]

# 입력과 출력 변수 설정
X_train = train_data.drop(['BMI', 'ID', '나이', '휴식 심박수', '평균 심박수'], axis=1)
y_train = train_data['BMI']
X_test = test_data.drop(['BMI', 'ID', '나이', '휴식 심박수', '평균 심박수'], axis=1)
y_test = test_data['BMI']

# 데이터 정규화 (SGD는 스케일링이 중요함)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SGD 회귀 모델 설정 및 훈련
sgd_model = SGDRegressor(max_iter=1000, tol=1e-3, learning_rate='optimal', random_state=42)
sgd_model.fit(X_train_scaled, y_train)

# 예측 및 성능 평가
y_pred = sgd_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# 데이터 불러오기
file_path = '/content/drive/MyDrive/1차 플젝/건강데이터_키_몸무게_삭제.csv'
data = pd.read_csv(file_path, encoding='cp949')

# 필요 없는 날짜 컬럼 삭제
columns_to_drop = ['날짜']
data = data.drop(columns=columns_to_drop)

# 범주형 변수 원-핫 인코딩
data_encoded = pd.get_dummies(data, columns=['성별', '운동 유형', '강도', '건강 상태', '흡연 상태'], drop_first=True)

# ID 기준으로 학습/테스트 데이터 분리
train_data = data_encoded[data_encoded['ID'] <= 2500]
test_data = data_encoded[(data_encoded['ID'] > 2500) & (data_encoded['ID'] <= 3000)]

# 입력과 출력 변수 설정
X_train = train_data.drop(['BMI', 'ID', '나이', '휴식 심박수', '평균 심박수'], axis=1)
y_train = train_data['BMI']
X_test = test_data.drop(['BMI', 'ID', '나이', '휴식 심박수', '평균 심박수'], axis=1)
y_test = test_data['BMI']

# 데이터 정규화 (SGD는 스케일링이 중요함)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SGD 회귀 모델 설정 및 훈련
sgd_model = SGDRegressor(max_iter=1000, tol=1e-3, learning_rate='optimal', random_state=42)
sgd_model.fit(X_train_scaled, y_train)

# 예측 및 성능 평가
y_pred = sgd_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}, R²: {r2}')

MSE: 8.054351953398216, R²: 0.3159215060445846


In [ ]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 데이터 불러오기
file_path = '/content/drive/MyDrive/1차 플젝/건강데이터_키_몸무게_삭제.csv'
data = pd.read_csv(file_path, encoding='cp949')

# 필요 없는 날짜 컬럼 삭제
columns_to_drop = ['날짜']
data = data.drop(columns=columns_to_drop)

# ID 기준으로 학습/테스트 데이터 분리
train_data = data[data['ID'] <= 2500]
test_data = data[(data['ID'] > 2500) & (data['ID'] <= 3000)]

# 입력과 출력 변수 설정
X_train = train_data.drop(['BMI', 'ID'], axis=1)
y_train = train_data['BMI']
X_test = test_data.drop(['BMI', 'ID'], axis=1)
y_test = test_data['BMI']

# 범주형 변수 이름 가져오기
categorical_features = ['성별', '운동 유형', '강도', '건강 상태', '흡연 상태']

# 결측값 처리 (필요 시)
for col in categorical_features:
    X_train[col] = X_train[col].fillna('missing')
    X_test[col] = X_test[col].fillna('missing')

# CatBoost 회귀 모델 설정 및 훈련
catboost_model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.1, verbose=200)
catboost_model.fit(X_train, y_train, cat_features=categorical_features)

# 예측 및 성능 평가
y_pred = catboost_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 데이터 불러오기
file_path = '/content/drive/MyDrive/1차 플젝/건강데이터_키_몸무게_삭제.csv'
data = pd.read_csv(file_path, encoding='cp949')

# 필요 없는 날짜 컬럼 삭제
columns_to_drop = ['날짜']
data = data.drop(columns=columns_to_drop)

# ID 기준으로 학습/테스트 데이터 분리
train_data = data[data['ID'] <= 2500]
test_data = data[(data['ID'] > 2500) & (data['ID'] <= 3000)]

# 입력과 출력 변수 설정
X_train = train_data.drop(['BMI', 'ID'], axis=1)
y_train = train_data['BMI']
X_test = test_data.drop(['BMI', 'ID'], axis=1)
y_test = test_data['BMI']

# 범주형 변수 이름 가져오기
categorical_features = ['성별', '운동 유형', '강도', '건강 상태', '흡연 상태']

# 결측값 처리 (필요 시)
for col in categorical_features:
    X_train[col] = X_train[col].fillna('missing')
    X_test[col] = X_test[col].fillna('missing')

# CatBoost 회귀 모델 설정 및 훈련
catboost_model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.1, verbose=200)
catboost_model.fit(X_train, y_train, cat_features=categorical_features)

# 예측 및 성능 평가
y_pred = catboost_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}, R²: {r2}')

0:	learn: 3.5320071	total: 638ms	remaining: 10m 37s
200:	learn: 2.3085996	total: 2m 7s	remaining: 8m 26s
400:	learn: 1.8288791	total: 4m 20s	remaining: 6m 28s
600:	learn: 1.5627766	total: 6m 26s	remaining: 4m 16s
800:	learn: 1.3686542	total: 8m 39s	remaining: 2m 8s
999:	learn: 1.1929099	total: 10m 44s	remaining: 0us
0:	learn: 3.5320071	total: 1.14s	remaining: 19m 1s
200:	learn: 2.3085996	total: 2m 7s	remaining: 8m 26s
400:	learn: 1.8288791	total: 4m 13s	remaining: 6m 19s
600:	learn: 1.5627766	total: 6m 22s	remaining: 4m 13s
800:	learn: 1.3686542	total: 8m 28s	remaining: 2m 6s
999:	learn: 1.1929099	total: 10m 34s	remaining: 0us
MSE: 3.2294291363028584, R²: 0.7257156090670064


In [ ]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 데이터 불러오기
file_path = '/content/drive/MyDrive/1차 플젝/건강데이터_키_몸무게_삭제.csv'
data = pd.read_csv(file_path, encoding='cp949')

# 필요 없는 날짜 컬럼 삭제
columns_to_drop = ['날짜']
data = data.drop(columns=columns_to_drop)

# ID 기준으로 학습/테스트 데이터 분리
train_data = data[data['ID'] <= 2500]
test_data = data[(data['ID'] > 2500) & (data['ID'] <= 3000)]

# 입력과 출력 변수 설정
X_train = train_data.drop(['BMI', 'ID'], axis=1)
y_train = train_data['BMI']
X_test = test_data.drop(['BMI', 'ID'], axis=1)
y_test = test_data['BMI']

# 범주형 변수 이름 가져오기
categorical_features = ['성별', '운동 유형', '강도', '건강 상태', '흡연 상태']

# 결측값 처리 (필요 시)
for col in categorical_features:
    X_train[col] = X_train[col].fillna('missing')
    X_test[col] = X_test[col].fillna('missing')

# CatBoost 회귀 모델 설정 및 훈련
catboost_model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.1, verbose=200)
catboost_model.fit(X_train, y_train, cat_features=categorical_features)

# 예측 및 성능 평가
y_pred = catboost_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 데이터 불러오기
file_path = '/content/drive/MyDrive/1차 플젝/건강데이터_키_몸무게_삭제.csv'
data = pd.read_csv(file_path, encoding='cp949')

# 필요 없는 날짜 컬럼 삭제
columns_to_drop = ['날짜']
data = data.drop(columns=columns_to_drop)

# ID 기준으로 학습/테스트 데이터 분리
train_data = data[data['ID'] <= 2500]
test_data = data[(data['ID'] > 2500) & (data['ID'] <= 3000)]

# 입력과 출력 변수 설정
X_train = train_data.drop(['BMI', 'ID',  '나이', '휴식 심박수'], axis=1)
y_train = train_data['BMI']
X_test = test_data.drop(['BMI', 'ID',  '나이', '휴식 심박수'], axis=1)
y_test = test_data['BMI']

# 범주형 변수 이름 가져오기
categorical_features = ['성별', '운동 유형', '강도', '건강 상태', '흡연 상태']

# 결측값 처리 (필요 시)
for col in categorical_features:
    X_train[col] = X_train[col].fillna('missing')
    X_test[col] = X_test[col].fillna('missing')

# CatBoost 회귀 모델 설정 및 훈련
catboost_model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.1, verbose=200)
catboost_model.fit(X_train, y_train, cat_features=categorical_features)

# 예측 및 성능 평가
y_pred = catboost_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}, R²: {r2}')

0:	learn: 3.5320071	total: 634ms	remaining: 10m 33s
200:	learn: 2.3085996	total: 2m 8s	remaining: 8m 32s
400:	learn: 1.8288791	total: 4m 17s	remaining: 6m 24s
600:	learn: 1.5627766	total: 6m 23s	remaining: 4m 14s
800:	learn: 1.3686542	total: 8m 30s	remaining: 2m 6s
999:	learn: 1.1929099	total: 10m 39s	remaining: 0us
0:	learn: 3.5327963	total: 615ms	remaining: 10m 14s
200:	learn: 2.3226328	total: 2m 8s	remaining: 8m 32s
400:	learn: 1.8800816	total: 4m 15s	remaining: 6m 22s
600:	learn: 1.6348418	total: 6m 22s	remaining: 4m 13s
800:	learn: 1.4834138	total: 8m 27s	remaining: 2m 6s
999:	learn: 1.3738052	total: 10m 33s	remaining: 0us
MSE: 2.826098133756988, R²: 0.7599716319455072


In [ ]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 데이터 불러오기
file_path = '/content/drive/MyDrive/1차 플젝/건강데이터_키_몸무게_삭제.csv'
data = pd.read_csv(file_path, encoding='cp949')

# 필요 없는 날짜 컬럼 삭제
columns_to_drop = ['날짜']
data = data.drop(columns=columns_to_drop)

# ID 기준으로 학습/테스트 데이터 분리
train_data = data[data['ID'] <= 2500]
test_data = data[(data['ID'] > 2500) & (data['ID'] <= 3000)]

# 입력과 출력 변수 설정
X_train = train_data.drop(['BMI', 'ID'], axis=1)
y_train = train_data['BMI']
X_test = test_data.drop(['BMI', 'ID'], axis=1)
y_test = test_data['BMI']

# 범주형 변수 이름 가져오기
categorical_features = ['성별', '운동 유형', '강도', '건강 상태', '흡연 상태']

# 결측값 처리 (필요 시)
for col in categorical_features:
    X_train[col] = X_train[col].fillna('missing')
    X_test[col] = X_test[col].fillna('missing')

# CatBoost 회귀 모델 설정 및 훈련
catboost_model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.1, verbose=200)
catboost_model.fit(X_train, y_train, cat_features=categorical_features)

# 예측 및 성능 평가
y_pred = catboost_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 데이터 불러오기
file_path = '/content/drive/MyDrive/1차 플젝/건강데이터_키_몸무게_삭제.csv'
data = pd.read_csv(file_path, encoding='cp949')

# 필요 없는 날짜 컬럼 삭제
columns_to_drop = ['날짜']
data = data.drop(columns=columns_to_drop)

# ID 기준으로 학습/테스트 데이터 분리
train_data = data[data['ID'] <= 2500]
test_data = data[(data['ID'] > 2500) & (data['ID'] <= 3000)]

# 입력과 출력 변수 설정
X_train = train_data.drop(['BMI', 'ID',  '나이', '휴식 심박수'], axis=1)
y_train = train_data['BMI']
X_test = test_data.drop(['BMI', 'ID',  '나이', '휴식 심박수'], axis=1)
y_test = test_data['BMI']

# 범주형 변수 이름 가져오기
categorical_features = ['성별', '운동 유형', '강도', '건강 상태', '흡연 상태']

# 결측값 처리 (필요 시)
for col in categorical_features:
    X_train[col] = X_train[col].fillna('missing')
    X_test[col] = X_test[col].fillna('missing')

# CatBoost 회귀 모델 설정 및 훈련
catboost_model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.1, verbose=200)
catboost_model.fit(X_train, y_train, cat_features=categorical_features)

# 예측 및 성능 평가
y_pred = catboost_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}, R²: {r2}')

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# 예측 및 실제 값
y_pred = catboost_model.predict(X_test)

# 지표 계산
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")


MSE: 2.8261
RMSE: 1.6811
MAE: 1.3371
R²: 0.7600


결과 해석:
MSE (2.8261)
→ 평균 제곱 오차가 2.8261로 나타났습니다. MSE는 오차를 제곱해 계산하기 때문에 큰 오차에 민감합니다. 따라서 MSE가 높을수록 예측의 정확도가 떨어지지만, 비교 지표로 사용하기 적합합니다.

RMSE (1.6811)
→ 평균 제곱근 오차가 1.6811로 나타났습니다. 실제 값과 예측 값의 평균적인 오차가 약 1.68 단위라는 의미입니다. RMSE는 MAE보다 큰 오차에 민감하며, 예측 오류를 직관적으로 파악할 수 있는 지표입니다.

MAE (1.3371)
→ 평균 절대 오차가 1.3371로 나타났습니다. 이는 평균적으로 예측 값과 실제 값이 약 1.34 단위만큼 차이 난다는 뜻입니다.

MAE는 RMSE보다 작은 오차에 민감하기 때문에 실제 예측 정확도를 보완하는 데 유용합니다.
R² (0.7600)
→ 결정계수 0.76은 모델이 약 76%의 데이터 변동성을 설명하고 있다는 의미입니다.

0.76은 회귀 모델에서 꽤 좋은 성능으로 볼 수 있지만, 여전히 약 24%의 변동성을 설명하지 못하고 있습니다.


---

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 데이터 불러오기
file_path = '/content/drive/MyDrive/1차 플젝/건강데이터_키_몸무게_삭제.csv'
data = pd.read_csv(file_path, encoding='cp949')

# 필요 없는 날짜 컬럼 삭제
columns_to_drop = ['날짜']
data = data.drop(columns=columns_to_drop)

# ID 기준으로 학습/테스트 데이터 분리
train_data = data[data['ID'] <= 2500]
test_data = data[(data['ID'] > 2500) & (data['ID'] <= 3000)]

# 입력과 출력 변수 설정
X_train = train_data.drop(['BMI', 'ID',  '나이', '휴식 심박수'], axis=1)
y_train = train_data['BMI']
X_test = test_data.drop(['BMI', 'ID',  '나이', '휴식 심박수'], axis=1)
y_test = test_data['BMI']

# 범주형 변수 이름 가져오기
categorical_features = ['성별', '운동 유형', '강도', '건강 상태', '흡연 상태']

# 결측값 처리 (필요 시)
for col in categorical_features:
    X_train[col] = X_train[col].fillna('missing')
    X_test[col] = X_test[col].fillna('missing')

# CatBoost 회귀 모델 설정 및 훈련
catboost_model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.1, verbose=200)
catboost_model.fit(X_train, y_train, cat_features=categorical_features)

# 예측 및 성능 평가
y_pred = catboost_model.predict(X_test)

# 지표 계산
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")

print(f'MSE: {mse}, R²: {r2}')

0:	learn: 3.5327963	total: 645ms	remaining: 10m 44s
200:	learn: 2.3226328	total: 2m 19s	remaining: 9m 16s
400:	learn: 1.8800816	total: 4m 27s	remaining: 6m 40s
600:	learn: 1.6348418	total: 6m 41s	remaining: 4m 26s
800:	learn: 1.4834138	total: 8m 53s	remaining: 2m 12s
999:	learn: 1.3738052	total: 11m 1s	remaining: 0us
MSE: 2.826098133756988
RMSE: 1.6811002747477581
MAE: 1.3371347523840684
R²: 0.7599716319455072
MSE: 2.826098133756988, R²: 0.7599716319455072


---

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from catboost import CatBoostRegressor, CatBoostClassifier # Import CatBoostClassifier
from sklearn.metrics import accuracy_score # Import accuracy_score

# 데이터 불러오기
file_path = '/content/drive/MyDrive/1차 플젝/건강데이터_bmi_분류.csv'
data = pd.read_csv(file_path, encoding='cp949')

# 필요 없는 날짜 컬럼 삭제
columns_to_drop = ['날짜']
data = data.drop(columns=columns_to_drop)

# ID 기준으로 학습/테스트 데이터 분리
train_data = data[data['ID'] <= 2500]
test_data = data[(data['ID'] > 2500) & (data['ID'] <= 3000)]

# 입력과 출력 변수 설정
X_train = train_data.drop(['BMI', 'ID',  '나이', '휴식 심박수', 'BMI 분류'], axis=1)
y_train = train_data['BMI 분류']
X_test = test_data.drop(['BMI', 'ID',  '나이', '휴식 심박수', 'BMI 분류'], axis=1)
y_test = test_data['BMI 분류']

# 범주형 변수 이름 가져오기
categorical_features = ['성별', '운동 유형', '강도', '건강 상태', '흡연 상태']

# 결측값 처리 (필요 시)
for col in categorical_features:
    X_train[col] = X_train[col].fillna('missing')
    X_test[col] = X_test[col].fillna('missing')

# CatBoost 분류 모델 설정 및 훈련
catboost_model = CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.1, verbose=200) # Changed to CatBoostClassifier
catboost_model.fit(X_train, y_train, cat_features=categorical_features)

# 예측 및 성능 평가
y_pred = catboost_model.predict(X_test)

# 분류 모델에 적합한 지표 계산 (예: 정확도)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

0:	learn: 1.3447095	total: 4.11s	remaining: 1h 8m 22s
200:	learn: 0.8739174	total: 15m 24s	remaining: 1h 1m 14s
400:	learn: 0.7164448	total: 30m 30s	remaining: 45m 34s
600:	learn: 0.6066070	total: 45m 38s	remaining: 30m 17s
800:	learn: 0.5143941	total: 1h 47s	remaining: 15m 6s
999:	learn: 0.4418360	total: 1h 15m 55s	remaining: 0us
Accuracy: 0.5916599657378596


In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

# 데이터 불러오기
file_path = '/content/drive/MyDrive/1차 플젝/건강데이터_bmi_분류.csv'
data = pd.read_csv(file_path, encoding='cp949')

# 필요 없는 날짜 컬럼 삭제
columns_to_drop = ['날짜']
data = data.drop(columns=columns_to_drop)

# ID 기준으로 학습/테스트 데이터 분리
train_data = data[data['ID'] <= 2500]
test_data = data[(data['ID'] > 2500) & (data['ID'] <= 3000)]

# 입력과 출력 변수 설정
X_train = train_data.drop(['BMI', 'ID', '나이', '휴식 심박수', 'BMI 분류'], axis=1)
y_train = train_data['BMI 분류']
X_test = test_data.drop(['BMI', 'ID', '나이', '휴식 심박수', 'BMI 분류'], axis=1)
y_test = test_data['BMI 분류']

# 범주형 특성을 숫자형으로 변환 (Ordinal Encoding for X_train and X_test)
categorical_features = ['성별', '운동 유형', '강도', '건강 상태', '흡연 상태']
encoder = OrdinalEncoder() # Create an OrdinalEncoder object

# Fit the encoder on the training data and transform both train and test
X_train[categorical_features] = encoder.fit_transform(X_train[categorical_features])
X_test[categorical_features] = encoder.transform(X_test[categorical_features])

# 범주형 특성을 숫자형으로 변환 (Label Encoding for y_train and y_test - you already have this)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# XGBoost 분류 모델 설정 및 훈련
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)  # Initialize the model
xgb_model.fit(X_train, y_train)  # Fit the model to the training data


# 예측 및 성능 평가
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))


Accuracy: 0.4958832989546551
              precision    recall  f1-score   support

           0       0.28      0.04      0.07     21688
           1       0.46      0.59      0.52     29205
           2       0.00      0.00      0.00     12718
           3       0.52      0.76      0.62     50801

    accuracy                           0.50    114412
   macro avg       0.32      0.35      0.30    114412
weighted avg       0.40      0.50      0.42    114412



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report

# 데이터 불러오기 및 전처리 동일

# LightGBM 분류 모델 설정 및 훈련
lgbm_model = LGBMClassifier(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42)
lgbm_model.fit(X_train, y_train)

# 예측 및 성능 평가
y_pred = lgbm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.084692 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1611
[LightGBM] [Info] Number of data points in the train set: 573289, number of used features: 15
[LightGBM] [Info] Start training from score -1.702232
[LightGBM] [Info] Start training from score -1.330811
[LightGBM] [Info] Start training from score -2.181165
[LightGBM] [Info] Start training from score -0.819729
Accuracy: 0.5595654301996295
              precision    recall  f1-score   support

           0       0.28      0.17      0.21     21688
           1       0.63      0.71      0.66     29205
           2       0.35      0.06      0.11     12718
           3       0.59      0.77      0.67     50801

    accuracy                           0.56    11

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Random Forest 분류 모델 설정 및 훈련
rf_model = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# 예측 및 성능 평가
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))


Accuracy: 0.49328741740376886
              precision    recall  f1-score   support

           0       0.33      0.01      0.03     21688
           1       0.44      0.59      0.50     29205
           2       0.00      0.00      0.00     12718
           3       0.53      0.76      0.62     50801

    accuracy                           0.49    114412
   macro avg       0.32      0.34      0.29    114412
weighted avg       0.41      0.49      0.41    114412

